# Initialize Session
Run the following pipeline from a subdirectory of the XMC run directory (where all the deconvolution files are).
(The thousands of files in the main run directory slows things down.)

In [ ]:
%run ../../diagnostic_init.py
%load_ext autoreload
%autoreload 2 #automatically reload functions before execution
outroot = './'
# Set up plotting to notebook
bplt.output_notebook()

# edit blobcols depending on specific model for this run
with open('../parameters.txt') as f:
    allcols = f.readlines()[0].strip('\n')
allcols = list(allcols.split(','))
print allcols
blobcols = [c for c in allcols if 'blob' in c]
print blobcols

# if level 1 cleaning already has been done, read in cleaned blobs, changing iteration numbers. 
# Skip the xd.clean below
dfall = pd.read_table('deconvolution_merged_iter3500-9292.txt',index_col=0,sep=r'\s+',engine='python')
#print dfall.columns

# Check Run Progress and Filter Data
Skip if already done and merged deconvolution file is being used.

In [ ]:
sf=xplt.chi2(runpath='../',outfile='notebook')

Set minimum converged iteration, based on chi2 plot.
Change distance depending on the object.


In [ ]:
itermin = 3500
# determine converged chi2/dof
sf = xw.filterblobs(sf,'iteration',minvals=itermin)
print xw.weighted_median(sf['redchi2']) 
print max(sf.iteration)

In [ ]:
# remove iterations before convergence, add new derived columns, 
#  and save merged deconvolution file
#dfall = xd.clean(runpath='../',itmin=itermin,distance=3.3)

# Get Basic Blob Info
# print columns
print dfall.columns
# print number of blobs
print len(dfall.index)
# print min, max of each parameter
for p in dfall.columns:
    print p,'\t',min(dfall[p]),'\t',max(dfall[p])

# Initial Inspection

In [ ]:
# Plot blob histograms
hfigs=xplt.histogram_grid(dfall,outfile='notebook',ncols=2,bins=200,height=200)
whfigs=xplt.histogram_grid(dfall,outfile='notebook',ncols=2,weights=dfall['blob_em'],bins=200,height=200)

It looks like almost all of the EM is in the lowest temperature bins ... 

In [ ]:
# Plot traceplots
# if Datashader not available, or linked plots are desired, set agg=None. Note this 
#  will be MUCH slower if too many parameters..
tfigs = xplt.traceplots(dfall[blobcols],agg=None,sampling=2000,outfile='notebook')

# Remove high EM - low kT blobs
Use the histograms and scatter plots to determine good guesses for an EM or temperature threshold.

Then create EM maps signficance maps for the 'bad' blobs to see if the emission from those blobs was significant.

In [ ]:
interesting_cols = ['blob_kT','blob_em','blob_nH','blob_sigma','blob_tau']
#tfigs2 = xplt.traceplots(dfall[interesting_cols],agg='dscount',sampling=1000,outfile='notebook')
#tfigs2 = xplt.traceplots(dfall[blobcols],agg=None,sampling=1000,outfile='notebook')
#sfigs = xplt.scatter(dfall,'blob_em','blob_kT',outfile='notebook',ylog='auto',agg=None,size=10)
xplt.histogram_grid(dfall[interesting_cols],outfile='notebook',bins=1000)

In [ ]:
#dfall['blob_solmass'] = astro.em_to_mass(dfall.blob_em,dfall.blob_volume,tounit='sol')
#xplt.histogram(dfall['blob_solmass'],outfile='notebook',bins=500)
print dfall.columns

### Create EM maps to check significance of blobs

In [ ]:
# Create EM map of all blobs for comparison
# change x0, y0, imagesize, binsize as appropriate (see the start_xmc file for x0,y0)
imgs=xm.make_map(dfall,paramname='blob_em',paramweights=None,iteration_type='total',
                 binsize=6.0,nlayers=20,imagesize=500.,withsignificance=True,nproc=4,
                 outfile=outroot+'bin6_500arcsec_all',x0=-51.,y0=63.,clobber=True,rotation=-90.0)

In [ ]:
# based on plots, set EM and/or kT threshold for 'bad' blobs
kTthresh = 0.13

# Filter blobs based on EM and/or kT
# change parameter name and limits as needed
dfbadkT = xw.filterblobs(dfall,'blob_kT',maxvals=kTthresh)

# Create EM map of 'bad' blobs
# change x0, y0, imagesize, binsize as appropriate (see the start_xmc file for x0,y0)
imgs2=xm.make_map(dfbadkT,paramname=['blob_em','blob_kT'],paramweights=[None,'blob_em'],iteration_type=['total','median'],
                 binsize=6.0,nlayers=20,imagesize=500.,withsignificance=True,nproc=6,
                 outfile=outroot+'bin6_500arcsec_badkT',x0=-51.,y0=63.,clobber=True,rotation=-90.0)

xplt.histogram(dfbadkT['blob_em'],bins=500,outfile='notebook',height=200)

# Open file in ds9
!ds9 bin6_500arcsec_all_median_blob_em.fits -multiframe bin6_500arcsec_badkT_median_blob_em.fits -cmap rainbow -match colorbars

There seems to be a small amount of low kT emission (kT<~0.16 keV) that is always significanct (greater than 1sigma). However, the low kT emission with the highest EM is not significant. Will try cutting on EM instead (based on the low kT EM histogram above), as that seems to be better indicator.

In [ ]:
# based on plots, set EM and/or kT threshold for 'bad' blobs
emthresh = 0.5e58

# Filter blobs based on EM and/or kT
# change parameter name and limits as needed
dfbadEM = xw.filterblobs(dfall,'blob_em',minvals=emthresh)

# Create EM map of 'bad' blobs
# change x0, y0, imagesize, binsize as appropriate (see the start_xmc file for x0,y0)
imgs1=xm.make_map(dfbadEM,paramname='blob_em',paramweights=None,iteration_type='total',
                 binsize=10.0,nlayers=70,imagesize=500.,withsignificance=True,nproc=4,
                 outfile=outroot+'bin10_500arcsec_badem',x0=-51.,y0=63.,clobber=True,rotation=-90.0)

xplt.histogram(dfbadEM['blob_kT'],bins=500,outfile='notebook',height=200)

# Open file in ds9
!ds9 bin6_500arcsec_all_median_blob_em.fits -multiframe bin10_500arcsec_badem_median_blob_em.fits -cmap rainbow -match colorbars

With binsize of 6", there seems again to be a little bit of significant emission (just over 1sigma), but at very low kT (<0.13). But emission with both EM>1e59 and kT<0.13 keV is not significant. So likely need to filter on both.

However, using binsize of 10", can go down to EM~0.5e58 before finding significant emission.

So binsize matters. Because I'm wanting to estimate the signficance of blobs, it makes sense to use binsizes that are similar to (and not substantially smaller than) typical blob sizes for this run. In this case, blob sizes are as small as 5-10" in radius (10-20" in diameter), so binsize shouldn't be smaller than about 10".

In [ ]:
# test if any of the lower EM, but low kT blobs are insignificant as well.
kTthresh = 0.164
emthresh = 0.5e58 # based on EM test above, with 10" binsize

# Filter blobs based on EM and/or kT
# change parameter name and limits as needed
dfbad = xw.filterblobs(dfall,'blob_em',maxvals=emthresh)
dfbad = xw.filterblobs(dfbad,'blob_kT',maxvals=kTthresh)

# Create EM map of 'bad' blobs
# change x0, y0, imagesize, binsize as appropriate (see the start_xmc file for x0,y0)
imgs2=xm.make_map(dfbad,paramname='blob_em',paramweights=None,iteration_type='total',
                 binsize=10.0,nlayers=50,imagesize=500.,withsignificance=True,nproc=4,
                 outfile=outroot+'bin10_500arcsec_bad',x0=-51.,y0=63.,clobber=True,rotation=-90.0)

xplt.histogram_grid(dfbad[['blob_kT','blob_em']],bins=500,outfile='notebook',height=200)

# Open file in ds9
!ds9 bin6_500arcsec_all_median_blob_em.fits -multiframe bin10_500arcsec_bad_median_blob_em.fits -cmap rainbow -match colorbars

From all the blobs remaining after the initial EM cut:

No emission below kT=0.164 is more than 1sigma significant. Increasing this kT threshold any amount results in small amounts of signficant emission.

Cuts should be: 1) cut any blobs with EM>0.5e58, and 2) cut any remaining blobs with kT<0.164 keV.

In [ ]:
# Once decided on proper thresholds, filter the dataframe for further data exploration. Check that lowest
#  lowest temperature bins do not dominate, and that there is not an extra peak in the lowest kT bin.
kTthresh = 0.164
emthresh = 0.5e58
dfgood = xw.filterblobs(dfall,'blob_em',maxvals=emthresh)
dfgood = xw.filterblobs(dfgood,'blob_kT',minvals=kTthresh)

interesting_cols = ['blob_kT','blob_em','blob_nH','blob_sigma','blob_tau']

# check distributions and plots with filtered blobs
#xplt.traceplots(dfall[interesting_cols],agg=None,sampling=5000,outfile='notebook')
xplt.histogram_grid(dfgood[blobcols],outfile='notebook',bins=500)
xplt.histogram_grid(dfgood[blobcols],outfile='notebook',bins=500,weights=dfgood['blob_em'])

# Check high kT blobs

### Histograms

In [ ]:
# Plot blob histograms
hfigs=xplt.histogram(dfgood['blob_kT'],outfile='notebook',bins=500,height=400)
whfigs=xplt.histogram(dfgood['blob_kT'],outfile='notebook',weights=dfgood['blob_em'],bins=500,height=300)

In [ ]:
tfigs = xplt.traceplots(dfgood[blobcols],agg=None,sampling=2000,outfile='notebook')

In [ ]:
hfigs=xplt.histogram_grid(dfgood,outfile='notebook',ncols=2,bins=300)
whfigs=xplt.histogram_grid(dfgood,outfile='notebook',ncols=2,weights=dfgood['blob_em'],bins=300)

In [ ]:
EMthresh = 2e55

# Create EM map of low EM blobs
# change x0, y0, imagesize, binsize as appropriate (see the start_xmc file for x0,y0)
imgs2=xm.make_map(xw.filterblobs(dfgood,'blob_em',maxvals=emthresh),paramname='blob_em',paramweights=None,
                  iteration_type='total',
                 binsize=10.0,nlayers=30,imagesize=500.,withsignificance=True,nproc=4,
                 outfile=outroot+'bin10_500arcsec_lowEM',x0=-51.,y0=63.,clobber=True,rotation=-90.0)

# Open file in ds9
!ds9 bin6_500arcsec_all_median_blob_em.fits -multiframe bin10_500arcsec_lowEM_median_blob_em.fits -cmap rainbow -match colorbars

In [ ]:
kTthresh = 3.45

# Create EM map of high kT blobs
# change x0, y0, imagesize, binsize as appropriate (see the start_xmc file for x0,y0)
imgs2=xm.make_map(xw.filterblobs(dfgood,'blob_kT',minvals=kTthresh),paramname='blob_em',paramweights=None,
                  iteration_type='total',
                 binsize=10.0,nlayers=200,imagesize=500.,withsignificance=True,nproc=4,
                 outfile=outroot+'bin10_500arcsec_highkT',x0=-51.,y0=63.,clobber=True,rotation=-90.0)

# Open file in ds9
!ds9 bin6_500arcsec_all_median_blob_em.fits -multiframe bin10_500arcsec_highkT_median_blob_em.fits -cmap rainbow -match colorbars

Start getting significant emission at kT>=3.44. So set threshold at kT=3.45, and cut all blobs above this.

In [ ]:
# trim highest kT blobs (insignificant)
kTthresh=3.45
dfgood = xw.filterblobs(dfgood,'blob_kT',maxvals=3.45)
print "Number of good blobs = ",len(dfgood.index)

# Filter blobs by location

In [ ]:
imgs2=xm.make_map(dfgood,paramname='blob_em',paramweights=None,
                  iteration_type='total',
                 binsize=10.0,nlayers=30,imagesize=700.,withsignificance=True,nproc=4,
                 outfile=outroot+'bin10_700arcsec_good',x0=-51.,y0=63.,clobber=True,rotation=-90.0)

# Open file in ds9
!ds9 -multiframe bin10_700arcsec_*.fits -cmap rainbow -match colorbars

Determine a radius around the object outside of which the emission is <1sigma significant. Blobs' emission measure will be weighted to include the fraction of its total emission within this radius. This should eliminate or signficantly reduce the influence of edge and corner blob emission (where there are extremely few photons to constrain things) in the histograms.

### Calculate this fractional weight and add it as a column 
(This will take awhile!)

In [ ]:
x0=-51.
y0=63.
r0=340.
rotation=-90.
dfgood = xw.filtercircle(dfgood,r0=r0,x0=x0,y0=y0,logic='include',fraction=True,regname='rcw103_circ',
                     use_ctypes=True,parallel=True,nproc=5)

### Verify low significance of de-weighted emission

In [ ]:
# get fractions outside of the main circle (the emission being excluded)
dfgood['rcw103_circ_exc_fraction'] = 1.0-dfgood.rcw103_circ_inc_fraction
imgs2=xm.make_map(dfgood,
                  paramname='blob_em',paramweights='rcw103_circ_exc_fraction',iteration_type='total',
                 binsize=10.0,nlayers=30,imagesize=700.,withsignificance=True,nproc=5,
                 outfile=outroot+'bin10_700arcsec_circle_exc',x0=x0,y0=y0,clobber=True,
                  rotation=rotation)

# make equivalent map with the include weights
imgs2=xm.make_map(dfgood,
                  paramname='blob_em',paramweights='rcw103_circ_inc_fraction',iteration_type='total',
                 binsize=10.0,nlayers=30,imagesize=700.,withsignificance=True,nproc=5,
                 outfile=outroot+'bin10_700arcsec_circle_inc',x0=x0,y0=y0,clobber=True,
                  rotation=rotation)


# Open files in ds9
!ds9 -multiframe bin10_700arcsec_*.fits -cmap rainbow -match colorbars

# Save cleaned blob file.

In [ ]:
outfile = ('deconvolution_merged_iter'+str(int(min(dfall.iteration)))+'-'+str(int(max(dfall.iteration)))+'_cleaned.txt')

f = open(outfile,'w+')
f.write('# Cleaning criteria: \n')
f.write('# EM<0.5e58 \n')
f.write('# 0.164<kT<3.45 keV \n')
f.write('# rcw103_circ: x0= '+str(x0)+', y0='+str(y0)+', r0='+str(r0)+'\n')
dfgood.to_csv(f,sep='\t')
f.close()